<a href="https://colab.research.google.com/github/Oumayma-O/local_rag_agents/blob/main/rag_local_ollama_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up ollama server

In [ ]:
!apt update -qq && apt install -y -qq lshw

50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.


In [ ]:
! curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update -qq && sudo apt install ngrok -qq

deb https://ngrok-agent.s3.amazonaws.com buster main
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
ngrok is already the newest version (3.14.0).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.


In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 11868    0 11868    0     0   9720      0 --:--:--  0:00:01 --:--:--  9727
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
OLLAMA_PORT=8081
OLLAMA_HOST=f"http://0.0.0.0:{OLLAMA_PORT}"
OLLAMA_BASE_URL=f"http://localhost:{OLLAMA_PORT}"

import os

os.environ['OLLAMA_HOST'] = OLLAMA_HOST

In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [ ]:
!ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 87048bcd5521...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 87048bcd5521...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 87048bcd5521...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 87048bcd5521...   0% ▕▏ 1.3 MB/4.7 GB                  pulling manifest 
pulling 87048bcd5521...   0% ▕▏  20 MB/4.7 GB                  pulling manifest 
pulling 87048bcd5521...   1% ▕▏  32 MB/4.7 GB                  pulling manifest 
pulling 87048bcd5521...   1% ▕▏  40 MB/4.7 GB                  pulling manifest 
pulling 87048bcd5521...   1% ▕▏  51 MB/4.7 GB                  pulling manifest 
pulling 87048bcd5521...   1% ▕▏  57 MB/4.7 GB                  pulli

In [ ]:
%pip install --upgrade --quiet langchain langchain-community langchainhub langchain-chroma langchain-core langchain-experimental bs4 gpt4all pyngrok

# Making the server available on the internet

In [ ]:
# Open a ngrok tunnel to the HTTP server
from pyngrok import ngrok
import subprocess

try:
  ngrok.set_auth_token("2kHVwc1zyenYu2xEIALgDIfrNAn_4FaCiGj9X2mNQThJM8qzS")
  public_url = ngrok.connect(OLLAMA_PORT).public_url
  print(f" * ngrok tunnel '{public_url}' -> '{OLLAMA_BASE_URL}'")

  result = subprocess.run([
    'curl',
    '-X', 'GET',
    '-H', 'ngrok-skip-browser-warning: 1',
    public_url,
  ],
    capture_output = True,
    text = True).stdout
  print(result)
except Exception as e:
  print(e)

 * ngrok tunnel 'https://4772-35-203-160-158.ngrok-free.app' -> 'http://localhost:8081'
Ollama is running


# rag with history

In [ ]:
"""import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.llms import Ollama
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter"""

'import bs4\nfrom langchain.chains import create_history_aware_retriever, create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_chroma import Chroma\nfrom langchain_community.chat_message_histories import ChatMessageHistory\nfrom langchain_community.document_loaders import WebBaseLoader\nfrom langchain_core.chat_history import BaseChatMessageHistory\nfrom langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder\nfrom langchain_core.runnables.history import RunnableWithMessageHistory\nfrom langchain.llms import Ollama\nfrom langchain_community.embeddings import GPT4AllEmbeddings\nfrom langchain.embeddings import OllamaEmbeddings\nfrom langchain_text_splitters import RecursiveCharacterTextSplitter'

In [ ]:
"""llm = Ollama(model="llama3:instruct", temperature=0, base_url=OLLAMA_BASE_URL, num_ctx=2048)"""

'llm = Ollama(model="llama3:instruct", temperature=0, base_url=OLLAMA_BASE_URL, num_ctx=2048)'

In [ ]:
"""# 1. Load, chunk and index the contents of the documentation to create a retriever.
oembed = OllamaEmbeddings(base_url=OLLAMA_BASE_URL, model="nomic-embed-text")
gpt4allembed = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf")

loader = WebBaseLoader(
    web_paths=("https://man7.org/linux/man-pages/man1/journalctl.1.html",
               "https://man7.org/linux/man-pages/man1/timedatectl.1.html"),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=gpt4allembed)
retriever = vectorstore.as_retriever()

### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. If asked for a command, only return the full "
    "command that answers the query, do not add extra arguments "
    "that are unnecessary. Use one sentence maximum and keep "
    "the answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)"""

'# 1. Load, chunk and index the contents of the documentation to create a retriever.\noembed = OllamaEmbeddings(base_url=OLLAMA_BASE_URL, model="nomic-embed-text")\ngpt4allembed = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf")\n\nloader = WebBaseLoader(\n    web_paths=("https://man7.org/linux/man-pages/man1/journalctl.1.html",\n               "https://man7.org/linux/man-pages/man1/timedatectl.1.html"),\n)\ndocs = loader.load()\n\ntext_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)\nsplits = text_splitter.split_documents(docs)\nvectorstore = Chroma.from_documents(documents=splits, embedding=gpt4allembed)\nretriever = vectorstore.as_retriever()\n\n### Contextualize question ###\ncontextualize_q_system_prompt = (\n    "Given a chat history and the latest user question "\n    "which might reference context in the chat history, "\n    "formulate a standalone question which can be understood "\n    "without the chat history. Do NOT answer the q

In [ ]:
"""conversational_rag_chain.invoke(
    {"input": "Write the command to get last 20 logs"},
    config={
        "configurable": {"session_id": "jnkjn5"}
    },
)["answer"]"""

'conversational_rag_chain.invoke(\n    {"input": "Write the command to get last 20 logs"},\n    config={\n        "configurable": {"session_id": "jnkjn5"}\n    },\n)["answer"]'

In [ ]:
"""!OLLAMA_HOST=0.0.0.0:8081 ollama list"""

'!OLLAMA_HOST=0.0.0.0:8081 ollama list'

# rag-agent-testing-local

In [ ]:
%pip install llama-index langfuse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
  Attempting uninstall: anyio
    Found existing installation: anyio 3.7.1
    Uninstalling anyio-3.7.1:
      Successfully uninstalled anyio-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of

In [ ]:
%%capture --no-stderr
%pip install -U langchain tavily-python langgraph matplotlib langchain_community langchain-openai scikit-learn langchainhub langchain-ollama langchain-nomic nomic[local]

In [ ]:
LANGFUSE_PORT=3000
#LANGFUSE_HOST=f"http://0.0.0.0:{OLLAMA_PORT}"
LANGFUSE_BASE_URL=f"http://localhost:{LANGFUSE_PORT}"

import os

#os.environ['OLLAMA_HOST'] = OLLAMA_HOST

In [ ]:
# Open a ngrok tunnel to the HTTP server
from pyngrok import ngrok
import subprocess

try:
  ngrok.set_auth_token("2kHVwc1zyenYu2xEIALgDIfrNAn_4FaCiGj9X2mNQThJM8qzS")
  public_url = ngrok.connect(LANGFUSE_PORT).public_url
  print(f" * ngrok tunnel '{public_url}' -> '{LANGFUSE_BASE_URL}'")

  result = subprocess.run([
    'curl',
    '-X', 'GET',
    '-H', 'ngrok-skip-browser-warning: 1',
    public_url,
  ],
    capture_output = True,
    text = True).stdout
  print(result)
except Exception as e:
  print(e)

 * ngrok tunnel 'https://2023-35-203-160-158.ngrok-free.app' -> 'http://localhost:3000'


<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [ ]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    public_key="pk-lf-3d54950b-b0e6-429d-95a1-c6fdc85d851e",
    secret_key="sk-lf-ae25d9ce-31ee-462a-87f1-e7825db82906",
    #host="http://localhost:3000"
    host=LANGFUSE_BASE_URL
)


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool

# List of URLs to load documents from
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Initialize a text splitter with specified chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

# Split the documents into chunks
doc_splits = text_splitter.split_documents(docs_list)

# Add the document chunks to the "vector store" using NomicEmbeddings
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
    #embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever(k=4)

In [ ]:
retriever.invoke("What are the types of agent memory?")

[Document(metadata={'id': '0a6e4508-1e2e-4fce-8427-769b53b08fdf', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'}, page_content='Fig. 7. Comparison of AD, ED, source policy and RL^2 on environments that require memory and exploration. Only binary reward is assigned. The source policies are trained with A3C for "dark" environments and DQN for watermaze.(Image source: Laskin et al. 2023)\nComponent Tw

In [ ]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.

    Use the following documents to answer the question.

    If you don't know the answer, just say that you don't know.

    Use three sentences maximum and keep the answer concise:
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    url='https://7329-34-16-208-221.ngrok-free.app',

)

rag_chain = prompt | llm | StrOutputParser()

In [ ]:
rag_chain.invoke(
    {
        "question": "What are the types of agent memory?",
        "documents": retriever.invoke("What are the types of agent memory?"),
    }
)

'There are several types of memory in human brains and agent systems. They include sensory memory, short-term memory, long-term memory, and maximum inner product search (MIPS). Sensory memory retains impressions of sensory information, while short-term memory is restricted by the finite context window length of Transformer. Long-term memory is an external vector store that can be accessed via fast retrieval.'

# retrieval garder

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

# JSON
llm = ChatOllama(model="llama3.1",
                 format="json",
                 temperature=0)


prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()

In [ ]:
retrieval_grader.invoke(
    {
        "question": "What are the types of agent memory?",
        "document": retriever.invoke("What are the types of agent memory?"),
    }
)

{'score': 'yes'}

# multi doc grader

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

# Define the modified prompt template
prompt = PromptTemplate(
    template="""You are a grader assessing relevance of retrieved documents to a user question.

    Here are the retrieved documents:
    {documents}

    Here is the user question: {question}

    For each document, if it contains keywords related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals.

    Provide a binary score 'yes' or 'no' for each document to indicate whether the document is relevant to the question.
    Provide the scores as a JSON list with keys 'doc1', 'doc2', 'doc3', and 'doc4', corresponding to each document.

    Scores:
    {{
        "doc1": "yes/no",
        "doc2": "yes/no",
        "doc3": "yes/no",
        "doc4": "yes/no"
    }}
    """,
    input_variables=["question", "documents"],
)

llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    url='https://7329-34-16-208-221.ngrok-free.app',

)

# Define a function to generate the input text with multiple documents
def format_documents(doc_list):
    formatted_docs = ""
    for i, doc in enumerate(doc_list, start=1):
        formatted_docs += f"Document {i}: {doc}\n\n"
    return formatted_docs

# Example usage with a list of documents
documents = retriever.invoke("What are the types of agent memory?")  # Assuming retriever is defined
formatted_documents = format_documents(documents)

# Prepare the input for the prompt
input_data = {
    "question": "What are the types of agent memory?",
    "documents": formatted_documents
}

# Create the RAG chain
rag_chain = prompt | llm | StrOutputParser()

# Invoke the RAG chain
response = rag_chain.invoke(input_data)
print(response)


To determine the relevance of each document to the user question, I'll scan through each document and look for keywords related to "agent memory".

Document 1: This document talks about a comparison between different algorithms, but it doesn't mention agent memory explicitly. However, it does discuss the concept of memory in the context of LLM-powered agents. So, I'd say this is somewhat relevant.

Document 2: This document discusses generative agents and their ability to create believable simulacra of human behavior. It mentions a "memory stream" that records a comprehensive list of agents' experience in natural language. This seems directly related to the user question about types of agent memory.

Document 3: This is a table of contents for an article about LLM-powered autonomous agents, and it lists "Component Two: Memory" as one of the sections. This suggests that the article will discuss different types of memory relevant to agent systems.

Document 4: This document talks about s

In [ ]:
%pip install langchain_core

In [ ]:
%pip install langchain_community langchain_ollama

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_ollama import ChatOllama

from langchain_core.pydantic_v1 import BaseModel, Field


class TransferRelevanceGrader(BaseModel):
    """Binary score for relevance check on football transfer news."""

    binary_score: str = Field(
        description="The article is relevant to the user question, 'yes' or 'no'"
    )


# JSON
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    url='https://7329-34-16-208-221.ngrok-free.app',

)

structured_llm_grader = llm.with_structured_output(TransferRelevanceGrader)

prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.""",
    input_variables=["question", "document"],
)

structured_retrieval_grader = prompt | structured_llm_grader | JsonOutputParser()

In [ ]:
structured_retrieval_grader.invoke(
    {
        "question": "What are the types of agent memory?",
        "document": retriever.invoke("What are the types of agent memory?"),
    }
)